# 02_Data_Cleaning.ipynb

## 1. Identify & quantify missing values  
## 2. Define imputation strategies  
## 3. Detect & handle outliers


In [4]:
import pandas as pd

# Load raw data
df = pd.read_csv("../data/raw/transactions.csv")

# 1️⃣ Quantify missing values
print("Missing values per column:\n", df.isnull().sum())

# 2️⃣ Select only numeric columns for outlier detection
num_df = df.select_dtypes(include="number")

# 3️⃣ Compute IQR and flag outliers
Q1 = num_df.quantile(0.25)
Q3 = num_df.quantile(0.75)
IQR = Q3 - Q1

outliers = ((num_df < (Q1 - 1.5 * IQR)) | (num_df > (Q3 + 1.5 * IQR))).sum()
print("\nOutliers per numeric column (IQR method):\n", outliers)


Missing values per column:
 order_id       0
customer_id    0
order_date     0
price          0
quantity       0
dtype: int64

Outliers per numeric column (IQR method):
 order_id    0
price       0
quantity    0
dtype: int64


In [5]:
# 4️⃣ Define and apply imputation

# Convert order_date to datetime
df['order_date'] = pd.to_datetime(df['order_date'])

# Imputation rules
median_price = df['price'].median()
median_qty   = df['quantity'].median()

df['price']    = df['price'].fillna(median_price)
df['quantity'] = df['quantity'].fillna(median_qty)

# For customer_id, use a placeholder
df['customer_id'] = df['customer_id'].fillna('UNKNOWN')

# Forward‑fill any missing dates
df['order_date'] = df['order_date'].fillna(method='ffill')

# Verify no missing values remain
print("\nMissing values after imputation:\n", df.isnull().sum())



Missing values after imputation:
 order_id       0
customer_id    0
order_date     0
price          0
quantity       0
dtype: int64


/tmp/ipykernel_37659/851931473.py:17: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['order_date'] = df['order_date'].fillna(method='ffill')


In [6]:
# 5️⃣ Save cleaned data
import os

os.makedirs("../data/processed", exist_ok=True)
clean_path = "../data/processed/cleaned_transactions.csv"
df.to_csv(clean_path, index=False)

print(f"Cleaned data written to {clean_path}")


Cleaned data written to ../data/processed/cleaned_transactions.csv
